In [1]:
import numpy as np
import pennylane as qml
from pennylane import X, Y, Z, I
from kak_tools import identify_algebra, split_pauli_algebra

In [2]:
def close_and_split(gens, verbose=True):
    gens = [list(op.pauli_rep)[0] for op in gens] # Map generators from Operator to PauliWord
    dla = qml.lie_closure(gens, pauli=True)
    dla = [list(op)[0] for op in dla] # Map dla from PauliSentence to PauliWord
    np.random.shuffle(dla)
    sub_dlas = split_pauli_algebra(dla, verbose=verbose)
    return dla, sub_dlas

In [3]:
# Test with so(2n)
n = 10
gens = [X(w) @ X(w+1) for w in range(n-1)] + [Y(w) @ Y(w+1) for w in range(n-1)] + [Z(w) for w in range(n)]
dla, sub_dlas = close_and_split(gens)
identifiers = identify_algebra(sub_dlas, verbose=True)

Found 1 component with dimension 190.
Dimension of component: 190.
Component 0 can be one of the following:
so(20)


In [5]:
# Test with random generators
def make_random_semisimple_algebra(num_wires=5, num_gens=7, seed=None):
    rng = np.random.default_rng(seed=seed)
    failure = True
    while failure:
        words = rng.choice([I, X, Y, Z], replace=True, size=(num_gens, num_wires))
        gens = [qml.prod(*(P(w) for w, P in enumerate(word))) for word in words]
        dla, sub_dlas = close_and_split(gens, verbose=False)
        dims = [len(sub) for sub in sub_dlas]
        if 1 not in dims:
            failure = False
    return dla
    
num_wires = 7
num_gens = 10
seed = 285
gens = make_random_semisimple_algebra(num_wires, num_gens, seed=seed)
print(f"Max DLA dimension: {4**num_wires-1}")
dla, sub_dlas = close_and_split(gens)
identifiers = identify_algebra(sub_dlas, verbose=True)

Max DLA dimension: 16383
Found 1 component with dimension 255.
Dimension of component: 255.
Component 0 can be one of the following:
su(16)
